### selenium 패키지 가져오기 

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
# import undetected_chromedriver as webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from dataclasses import dataclass, asdict
import json

### 드라이버 옵션 설정

In [ ]:
options = webdriver.ChromeOptions()
options = Options()
### headless 모드 옵션
# options.add_argument('no-sandbox')
# options.add_argument('disable-dev-shm-usage')
# options.add_argument('headless')
# options.add_argument('disable-gpu')
### 세션 정보 저장 옵션(로그인 정보 등 페이지 기록 유지)
# chrome_profile_dir = 'my_session'
# options.add_argument(f'--user-data-dir={chrome_profile_dir}/')
driver = webdriver.Chrome(
    service=ChromeService(executable_path=ChromeDriverManager().install()),
    options=options
)

## youtube 스크립트 가져오기 

In [ ]:
driver.get('https://www.youtube.com/watch?v=vUmnCnGSIfA') #  ##나라장터 어떻게 사용하는 걸까? "나라장터 사용자 이용 가이드"
driver.implicitly_wait(10)

In [ ]:
button_path = '#movie_player > div.ytp-chrome-bottom > div.ytp-chrome-controls > div.ytp-left-controls > button'
play_button = driver.find_element(By.CSS_SELECTOR, button_path)
play_button.send_keys(Keys.ENTER)

In [ ]:
driver.find_element(By.CSS_SELECTOR, '#button-shape > button > yt-touch-feedback-shape > div > div.yt-spec-touch-feedback-shape__fill').click()
driver.find_element(By.CSS_SELECTOR, '#items > ytd-menu-service-item-renderer > tp-yt-paper-item').click()
time.sleep(1)

In [ ]:
segments_container = driver.find_element(By.CSS_SELECTOR, '#segments-container')

In [ ]:
segelem_list = segments_container.find_elements(By.XPATH, '//ytd-transcript-segment-renderer')
len(segelem_list)

In [ ]:
@dataclass
class ScriptItem:
    timestamp: str
    text: str

In [ ]:
script_list = []
for seg in segelem_list:
    # print(seg.text)
    e={}
    si = seg.text.find('\n')
    timestamp = seg.text[:si]
    text = seg.text[si+1:]
    # timestamp = seg.find_element(By.CLASS_NAME, 'segment-timestamp').text
    # text = seg.find_element(By.CLASS_NAME, 'segment-text').text
    # timestamp = seg.find_element(By.XPATH, '//div/div/div').text
    # text = seg.find_element(By.XPATH, '//div/yt-formatted-string').text
    e = ScriptItem(timestamp=timestamp, text=text)
    script_list.append(e)
    print(e)

### 스크립트 파일 저장

In [ ]:
dict_list = [asdict(e) for e in script_list]

In [ ]:
json_filename = 'script.json'
with open(json_filename, 'w', encoding='utf-8') as f:
    json.dump(dict_list, f, indent=4,ensure_ascii=False)

### driver 닫기

In [ ]:
driver.quit()